<a href="https://colab.research.google.com/github/komazawa-deep-learning/komazawa-deep-learning.github.io/blob/master/2021notebooks/2021_1105OpenAI_Gym_cartpole_rendering.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

 # 2021年度駒澤大学心理学特講 IIIB

 ## OpenAI の gym 環境を用いた強化学習デモ

- filename: `2021_1105OpenAI_Gym_cartpole_rendering.ipynb`
- author: 浅川伸一
- data: 2021_1105

- [OpenAI の開発した強化学習環境 gym](http://gym.openai.com/)
- [starai による Google Colab 上で Gym をレンダリングする環境 ](https://star-ai.github.io/Rendering-OpenAi-Gym-in-Colaboratory/) 

上記 2 つの環境整備により実習が可能となります。すぐれた環境を提供してくれたすべての関係者に感謝いたします。

In [ ]:
# もしかしたら，このセルの実行は不要かも 2020-0715 メモ
# !apt-get update > /dev/null 2>&1
# !apt-get install cmake > /dev/null 2>&1
# !pip install --upgrade setuptools 2>&1
# !pip install ez_setup > /dev/null 2>&1
#ATARI のゲーム環境を使いたければ直下行のコメントを外してください
!pip install gym[atari] > /dev/null 2>&1

# Gym とレンダリング環境が依存するライブラリのインストール


In [ ]:
#何が起こっているかを知りたい場合には下のコマンド中不等号以下をremove " > /dev/null 2>&1" を削除してください
!pip install gym pyvirtualdisplay > /dev/null 2>&1
!apt-get install -y xvfb python-opengl ffmpeg > /dev/null 2>&1

In [ ]:
# gym では，どのような環境が利用できるか表示します。
# 出力が長大になりますので，下までスクロールしてください。
from gym import envs
for e in envs.registry.all():
    print(e)

# 必要なヘルパ関数やライブラリをインポート


In [ ]:
import gym
from gym import logger as gymlogger
from gym.wrappers import Monitor
gymlogger.set_level(40) #error only
import tensorflow as tf
import numpy as np
import random
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline
import math
import glob
import io
import base64
from IPython.display import HTML

from IPython import display as ipythondisplay

In [ ]:
from pyvirtualdisplay import Display
display = Display(visible=0, size=(1400, 900))
display.start()

In [ ]:
"""
Gym 環境をビデオ撮影して表示するユーティリティ関数
ビデオを有効にするには "env = wrap_env(env) “ とするだけです。
"""

def show_video():
  mp4list = glob.glob('video/*.mp4')
  if len(mp4list) > 0:
    mp4 = mp4list[0]
    video = io.open(mp4, 'r+b').read()
    encoded = base64.b64encode(video)
    ipythondisplay.display(HTML(data='''<video alt="test" autoplay 
                loop controls style="height: 400px;">
                <source src="data:video/mp4;base64,{0}" type="video/mp4" />
             </video>'''.format(encoded.decode('ascii'))))
  else: 
    print("Could not find video")
    

def wrap_env(env):
  env = Monitor(env, './video', force=True)
  return env

# いろいろ遊んでみよう！

In [ ]:
env = wrap_env(gym.make("CartPole-v0"))

# 下のパックマン環境は ROM が必要です
# env = wrap_env(gym.make("MsPacman-v0"))

In [ ]:
# 上で選んだ環境の行動空間を表示してみましょう
print(env.action_space)

In [ ]:
observation = env.reset()

while True:
  
    env.render()
    
    #ここにエージェントの動作を書く必要があります
    action = env.action_space.sample() 
         
    observation, reward, done, info = env.step(action) 
   
        
    if done: 
      break;
            
env.close()
show_video()